In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics

# import the library

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import xml.etree.ElementTree as ET 
import cv2 as cv
import cv2 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# read the samples code

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(10,5))
image = cv.imread("/kaggle/input/face-expression-yolo-data/Face_Expressions_Images/Face_10.jpeg")
ax[0].imshow(image)
image = cv.imread("/kaggle/input/face-expression-yolo-data/Face_Expressions_Images/Face_2.jpg")
ax[1].imshow(image)
image = cv.imread("/kaggle/input/face-expression-yolo-data/Face_Expressions_Images/Face_27.jpg")
ax[2].imshow(image)
image = cv.imread("/kaggle/input/face-expression-yolo-data/Face_Expressions_Images/Face_35.jpeg")
ax[3].imshow(image)
fig.show()

In [ ]:
annotations_path = "/kaggle/input/face-expression-labels"
images_path = "/kaggle/input/face-expression-yolo-data/Face_Expressions_Images"

In [ ]:
dataset = {
            "file":[],
            "name":[],    
            "width":[],
            "height":[],
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
           }
for anno in glob.glob(annotations_path+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]]
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]
        

In [ ]:
dataset

In [ ]:
dataset['file'][0]

In [ ]:
df=pd.DataFrame(dataset)
df.head()

In [ ]:
df.name.unique()

# creating the class/target-converting categorical to numerical

In [ ]:
name_dict = {
    'Happy': 0,
    'Sad': 1,
    'Surprise': 2,
    'Angry':3
}

df['class'] = df['name'].map(name_dict)

In [ ]:
df.head()

In [ ]:
fileNames = [*os.listdir("/kaggle/input/face-expression-yolo-data/Face_Expressions_Images")]
print('There are {} images in the dataset'.format(len(fileNames)))

In [ ]:
fileNames

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(fileNames, test_size=0.2, random_state=22)
test, val = train_test_split(test, test_size=0.7, random_state=22)
print("Length of Train =",len(train))
print("="*30)
print("Length of Valid =",len(val))
print("="*30)
print("Length of test =", len(test))

In [ ]:
# create separate train test and validation dir inside yolo/data/
os.chdir('/kaggle/working/')
os.mkdir('./train')
os.mkdir('./val')
os.mkdir('./test')
os.mkdir('./train/images')
os.mkdir('./train/labels')
os.mkdir('./test/images')
os.mkdir('./test/labels')
os.mkdir('./val/images')
os.mkdir('./val/labels')

In [ ]:
input_data = '/kaggle/input/face-expression-yolo-data/Face_Expressions_Images/'
output_data = '/kaggle/working/'

In [ ]:
train

In [ ]:
from PIL import Image
def copyImages(imageList, folder_Name):
    for image in imageList:
        print(input_data+image)
        img = Image.open(input_data+image)
        img1 = img.resize((640, 480))
        print(output_data+folder_Name+"/images/"+image)
        _ = img1.save(output_data+folder_Name+"/images/"+image)

copyImages(train, "train")
copyImages(val, "val")
copyImages(test, "test")

In [ ]:
df['xmax'] = (640/df['width'])*df['xmax']
df['ymax'] = (480/df['height'])*df['ymax']
df['xmin'] = (640/df['width'])*df['xmin']
df['ymin'] = (480/df['height'])*df['ymin']

In [ ]:
df[['xmax', 'ymax', 'xmin', 'ymin']] = df[['xmax', 'ymax', 'xmin', 'ymin']].astype('int64')

In [ ]:
df['x_center'] = (df['xmax']+df['xmin'])/(2*640)
df['y_center'] = (df['ymax']+df['ymin'])/(2*480)
df['box_height'] = (df['xmax']-df['xmin'])/(640)
df['box_width'] = (df['ymax']-df['ymin'])/(480)

In [ ]:
df = df.astype('string')

In [ ]:
def create_labels(image_list, data_name):
    fileNames = [x.split(".")[0] for x in image_list]

    for name in fileNames:
        data = df[df.file==name]
        box_list = []
        
        for index in range(len(data)):
            row = data.iloc[index]
            box_list.append(row['class']+" "+row["x_center"]+" "+row["y_center"]\
                        +" "+row["box_height"]+" "+row["box_width"])
            
        text = "\n".join(box_list)
        with open(output_data+data_name+"/labels/"+name+".txt", "w") as file:
            file.write(text)


create_labels(train, "train")
create_labels(val, "val")
create_labels(test, "test")

# my data split/segrigation-training/testing /validation is done

In [ ]:
!ls

# load the pretrained model

In [ ]:
from ultralytics import YOLO

# load pre-trained model
detection_model = YOLO("yolov8m.pt")

In [ ]:
text = """
train: /kaggle/working/train
test: /kaggle/working/test
val: /kaggle/working/val

# number of classes
nc: 4

# class names
names: ['Happy', 'Surprise', 'Sad', 'Angry']
"""
with open("data.yaml", 'w') as file:
    file.write(text)

In [ ]:
!ls

# model-building /training-fine tune the model

In [ ]:
model = YOLO("yolov8m.pt")

model.train(data='data.yaml',epochs=5)

# model result display

In [ ]:
%matplotlib inline
model_path = 'runs/detect/train/'

def plot(ls, size):
    c=1
    plt.figure(figsize=(15,10))
    for im in ls:
        plt.subplot(size[0],size[1],c)
        im = plt.imread(model_path+im)
        plt.imshow(im)
        c+=1
    plt.show()

plot(['P_curve.png','R_curve.png'], (1,2))
plot(['F1_curve.png','PR_curve.png'], (1,2))
plot(['confusion_matrix.png','labels.jpg'], (1,2))
plot(['results.png'],(1,1))

In [ ]:
images_path = '/kaggle/working/test/images'

In [ ]:
import random
# choose random image from dataset
plt.figure(figsize=(20,20))
imgs = random.sample(os.listdir(images_path), 3)
c=1
for img in imgs:
    print(images_path+'/'+img)
    i=model.predict(source=images_path+'/'+img, conf=0.4, save=True, line_thickness=2)
    c+=1

In [ ]:
imgs_folders = random.sample(os.listdir('/kaggle/working/runs/detect'), 2)
imgs_folders

In [ ]:
os.listdir('/kaggle/working/runs/detect/')

In [ ]:
path = '/kaggle/working/runs/detect/train23/'
imgs = os.listdir(path)
plt.figure(figsize=(20,20))
c=1
for img in imgs:
    print(path+img)
    im = plt.imread(path+img)
    plt.subplot(2,3,c)
    plt.axis('off')
    plt.imshow(im)
    c+=1

In [ ]:
path = '/kaggle/working/runs/detect/train24/'
imgs = os.listdir(path)
plt.figure(figsize=(20,20))
c=1
for img in imgs:
    print(path+img)
    im = plt.imread(path+img)
    plt.subplot(2,3,c)
    plt.axis('off')
    plt.imshow(im)
    c+=1

In [ ]:
path = '/kaggle/working/runs/detect/train22/'
imgs = os.listdir(path)
plt.figure(figsize=(20,20))
c=1
for img in imgs:
    print(path+img)
    im = plt.imread(path+img)
    plt.subplot(2,3,c)
    plt.axis('off')
    plt.imshow(im)
    c+=1

In [ ]:
path = '/kaggle/working/runs/detect/train23/'
imgs = os.listdir(path)
plt.figure(figsize=(20,20))
c=1
for img in imgs:
    print(path+img)
    im = plt.imread(path+img)
    plt.subplot(2,3,c)
    plt.axis('off')
    plt.imshow(im)
    c+=1